In [ ]:
import torch, platform, sys

print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


In [ ]:
!pip install -U "transformers>=4.37.0" accelerate einops tiktoken \
    umap-learn scikit-learn matplotlib

# If you want a fresh torch, uncomment (Colab usually has a good version already)
# !pip install -U torch --index-url https://download.pytorch.org/whl/cu121


In [ ]:
import os

# Clone (skip if you already have it)
if not os.path.exists("protoInterpretation"):
    !git clone https://github.com/Nonempirical/protoInterpretation protoInterpretation

%cd protoInterpretation

# Editable install
!pip install -e .


In [ ]:
from src.protoInterpretation import (
    ModelWrapper,
    SamplingConfig,
    sample_chain,
    compute_horizon_metrics,
    project_step_embeddings,
    plot_entropy_curve,
    plot_horizon_width,
    plot_step_scatter_2d,
    save_batch_npz,
    save_metrics_json,
)
from src.protoInterpretation.model import HFModelAdapter, HFModelConfig

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get("hfKey")  # your Colab secret name
login(token=HF_TOKEN)


In [ ]:
model_name = "meta-llama/Meta-Llama-3.1-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

hf_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Note: we do not request attention weights during sampling, so we keep the default
# attention implementation (typically faster than forcing eager attention).

model = HFModelAdapter.__new__(HFModelAdapter)
model.config = HFModelConfig(
    model_name_or_path=model_name,
    dtype="bfloat16"
)
model.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.tokenizer = tokenizer
model.pad_token_id = tokenizer.pad_token_id
model.model = hf_model
model.tokenizer.padding_side = "left"


In [ ]:
from src.protoInterpretation import SamplingConfig  # adjust import if yours differs

prompts = [
    # --- OPEN ---
    "A bat is",
    "The woman in the blue dress",
    "I saw",
    "Something happens when there exists",
    "The man",
    "The man in the street",

    # --- CLOSED ---
    "A pencil is",
    "Napoleon is",
    "Photosynthesis is the process where",
    "The declaration of Independence, formally",
    "Photosynthesis is",
    "Erosion is",
]

cfg = SamplingConfig(
    num_chains=256,
    max_steps=32,
    temperature=0.9,
    top_k=0,
    top_p=0.9,
    seed=42,
    store_topk_logits=50,
    store_attention_weights=False,
)

In [ ]:
import os
import re
from datetime import datetime

USE_GOOGLE_DRIVE = True  # toggle

if USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    BASE_RUN_DIR = "/content/drive/MyDrive/protoInterpretation-runs"
else:
    BASE_RUN_DIR = "./runs"

from src.protoInterpretation import (
    sample_chain,
    compute_horizon_metrics,
    save_batch_npz,
    save_metrics_json,
)

def slugify_prompt(text: str, max_len: int = 60) -> str:
    """
    "The bat is in :" -> "the_bat_is_in"
    Safe for filenames. Truncates to max_len.
    """
    s = text.strip().lower()
    s = re.sub(r"[^a-z0-9]+", "_", s)     # non-alnum -> _
    s = re.sub(r"_+", "_", s).strip("_") # collapse + trim
    return (s[:max_len].rstrip("_")) or "empty_prompt"

def save_horizon_run_from_prompt(prompt_text: str, batch, metrics):
    os.makedirs(BASE_RUN_DIR, exist_ok=True)

    slug = slugify_prompt(prompt_text)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_id = f"{slug}_{timestamp}"   # e.g. the_bat_is_in_20251218_121530
    run_dir = os.path.join(BASE_RUN_DIR, run_id)
    os.makedirs(run_dir, exist_ok=True)

    save_batch_npz(batch, os.path.join(run_dir, "batch.npz"))
    save_metrics_json(metrics, os.path.join(run_dir, "metrics.json"), batch_meta=batch.meta)

    print(f"Saved → {run_dir}")

def run_prompts_and_save(model, prompts, cfg):
    for p in prompts:
        batch = sample_chain(model, p, cfg)
        metrics = compute_horizon_metrics(batch)
        save_horizon_run_from_prompt(p, batch, metrics)

# Run everything
run_prompts_and_save(model, prompts, cfg)
